In [1]:
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

2025/09/16 11:31:10 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/16 11:31:10 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


[<Experiment: artifact_location='/workspaces/Mlops-Zoomcamp-2025/02-experiment-tracking/mlruns/2', creation_time=1757655991553, experiment_id='2', last_update_time=1757655991553, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/Mlops-Zoomcamp-2025/02-experiment-tracking/mlruns/1', creation_time=1757654328038, experiment_id='1', last_update_time=1757654328038, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/Mlops-Zoomcamp-2025/02-experiment-tracking/mlruns/0', creation_time=1757654328030, experiment_id='0', last_update_time=1757654328030, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
client.create_experiment(name="my-next-experiment")

'3'

In [40]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='3',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [41]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: c2f4ec9b87874589a0933be346a56cac, rmse: 7.6166
run id: c94de1a1d7d54fba87e6582023251d7a, rmse: 7.6166
run id: 6ec0bfb5d0e3487aa67d86a5ea4fddb3, rmse: 7.6166
run id: d9d1d2ec955647bfb4bde16d168bb62d, rmse: 7.6166
run id: 7c26c473216245b2909af5ddec5cc339, rmse: 7.6166


Model Registry



In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [7]:
client.get_run("d011c6d504ae462bb4666d12900ee4d1")

<Run: data=<RunData: metrics={'rmse': 9.23343622815571}, params={'learning_rate': '0.09585355369315604',
 'max_depth': '30',
 'min_child_weight': '1.060597050922164',
 'objective': 'reg:linear',
 'reg_alpha': '0.018060244040060163',
 'reg_lambda': '0.011658731377413597',
 'seed': '42'}, tags={'mlflow.runName': 'masked-kit-244',
 'mlflow.source.name': '/home/codespace/.local/lib/python3.12/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'codespace'}>, info=<RunInfo: artifact_uri='/workspaces/Mlops-Zoomcamp-2025/02-experiment-tracking/mlruns/1/d011c6d504ae462bb4666d12900ee4d1/artifacts', end_time=1757654916717, experiment_id='1', lifecycle_stage='active', run_id='d011c6d504ae462bb4666d12900ee4d1', run_name='masked-kit-244', start_time=1757654715337, status='FINISHED', user_id='codespace'>, inputs=<RunInputs: dataset_inputs=[], model_inputs=[]>, outputs=<RunOutputs: model_outputs=[<LoggedModelOutput: model_id='m-1e0dc97aaca5467d910e3f0f4d249d08', step

In [8]:
run_id = "d011c6d504ae462bb4666d12900ee4d1"
model_uri = f"runs:/{run_id}/models_mlflow"  # Use the actual artifact name - models_mlflow
mlflow.register_model(model_uri=model_uri, name="nyc-taxi")

2025/09/16 11:31:53 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/16 11:31:53 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'nyc-taxi' already exists. Creating a new version of this model...
2025/09/16 11:31:53 WARNING mlflow.tracking._model_registry.fluent: Run with id d011c6d504ae462bb4666d12900ee4d1 has no artifacts at artifact path 'models_mlflow', registering model based on models:/m-1e0dc97aaca5467d910e3f0f4d249d08 instead
Created version '2' of model 'nyc-taxi'.


<ModelVersion: aliases=[], creation_timestamp=1758022313457, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1758022313457, metrics=None, model_id=None, name='nyc-taxi', params=None, run_id='d011c6d504ae462bb4666d12900ee4d1', run_link=None, source='models:/m-1e0dc97aaca5467d910e3f0f4d249d08', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [9]:
model_name = "nyc-taxi-model"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: None


/tmp/ipykernel_34233/29955185.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [24]:
versions = client.search_model_versions(f"name='{model_name}'")
for v in versions:
    print(f"Version: {v.version}, Current Stage: {v.current_stage}")

Version: 1, Current Stage: None
Version: 3, Current Stage: None
Version: 2, Current Stage: None


In [11]:
# Register a new model but there is no model_uri
# client.create_registered_model(name="your_new_model_name")
# Register a new version of an already existing model and run id
client.create_model_version(
    name=model_name,
    source=f"runs:/{run_id}/model"
)

<ModelVersion: aliases=[], creation_timestamp=1758022356529, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1758022356529, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id=None, run_link=None, source='runs:/d011c6d504ae462bb4666d12900ee4d1/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [85]:
model_version = 1
new_stage = "Staging"
# Instead of stages, use aliases (new way)
# Alias has replaced staging
client.set_registered_model_alias(
    name=model_name,
    alias="development",
    version=model_version,
   
)

In [86]:
client.transition_model_version_stage(
    name="nyc-taxi-lasso-model",
    version=1,
    stage="Staging"  # or "Production", "Archived", "None"
)

/tmp/ipykernel_34233/4094165861.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1758025715710, current_stage='Staging', deployment_job_state=None, description=None, last_updated_timestamp=1758030924878, metrics=None, model_id=None, name='nyc-taxi-lasso-model', params=None, run_id='c94de1a1d7d54fba87e6582023251d7a', run_link=None, source='models:/m-cd58a7a62b384ba4aedf01cf396bb1b5', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [ ]:
model_version = client.get_model_version(
     name="nyc-taxi-lasso-model",
    version=1,
)
print(model_version.current_stage)
# Get all versions of a model
# model = client.get_registered_model(name=model_name)
# for version in client.search_model_versions(f"name='{model_name}'"):
#     print(f"Version {version.version}: {version.current_stage}")

Staging


In [14]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=['development', 'staging'], creation_timestamp=1757658592311, current_stage='None', deployment_job_state=None, description='The model version 1 was transitioned to Staging on 2025-09-16', last_updated_timestamp=1758022569446, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='d011c6d504ae462bb4666d12900ee4d1', run_link=None, source='models:/m-1e0dc97aaca5467d910e3f0f4d249d08', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [22]:
client.search_registered_models()

# Check model versions
client.search_model_versions(f"name='{model_name}'")

[<ModelVersion: aliases=[], creation_timestamp=1757658592311, current_stage='None', deployment_job_state=None, description='The model version 1 was transitioned to Staging on 2025-09-16', last_updated_timestamp=1758022569446, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='d011c6d504ae462bb4666d12900ee4d1', run_link=None, source='models:/m-1e0dc97aaca5467d910e3f0f4d249d08', status='READY', status_message=None, tags={}, user_id=None, version=1>,
 <ModelVersion: aliases=[], creation_timestamp=1758022356529, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1758022356529, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id=None, run_link=None, source='runs:/d011c6d504ae462bb4666d12900ee4d1/model', status='READY', status_message=None, tags={}, user_id=None, version=3>,
 <ModelVersion: aliases=[], creation_timestamp=1757835095117, current_stage='None', deployment_job_state=None, description=None, last_updat

In [27]:
pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]3 [ipywidgets]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
client.list_artifacts("d011c6d504ae462bb4666d12900ee4d1")

[]

In [42]:
import mlflow.pyfunc

# Load model from Model Registry
mlflow.set_tracking_uri("sqlite:///mlflow.db")  # or wherever your MLflow UI is running
model = mlflow.pyfunc.load_model("runs:/c94de1a1d7d54fba87e6582023251d7a/model")

In [ ]:
from mlflow.tracking import MlflowClient
client = MlflowClient()

# List artifacts to see the correct path
artifacts = client.list_artifacts("c94de1a1d7d54fba87e6582023251d7a")
for artifact in artifacts:
    print(artifact.path)

# Then load with the correct path (usually "model")
model = mlflow.pyfunc.load_model("runs:/c94de1a1d7d54fba87e6582023251d7a/model")

models_pickle


In [45]:
model = mlflow.pyfunc.load_model(f"models:/nyc-taxi-lasso-model/1")
print(model)

mlflow.pyfunc.loaded_model:
  artifact_path: /workspaces/Mlops-Zoomcamp-2025/02-experiment-tracking/mlruns/3/models/m-cd58a7a62b384ba4aedf01cf396bb1b5/artifacts
  flavor: mlflow.sklearn
  run_id: c94de1a1d7d54fba87e6582023251d7a



In [16]:
from sklearn.metrics import root_mean_squared_error

In [48]:
import pandas as pd


In [49]:
pip install pyarrow fastparquet


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [50]:
import pyarrow
import fastparquet
print(f"PyArrow version: {pyarrow.__version__}, {fastparquet.__version__}")

PyArrow version: 21.0.0, 2024.11.0


In [51]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename, engine='fastparquet')

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(run_id, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"runs:/{run_id}/model")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [52]:
df = read_dataframe("data/green_tripdata_2021-04.parquet")

In [ ]:
model = mlflow.pyfunc.load_model("runs:/c94de1a1d7d54fba87e6582023251d7a/model")

In [55]:
print(model)

mlflow.pyfunc.loaded_model:
  artifact_path: /workspaces/Mlops-Zoomcamp-2025/02-experiment-tracking/mlruns/3/models/m-cd58a7a62b384ba4aedf01cf396bb1b5/artifacts
  flavor: mlflow.sklearn
  run_id: c94de1a1d7d54fba87e6582023251d7a



In [65]:
model = mlflow.pyfunc.load_model(f"runs:/c94de1a1d7d54fba87e6582023251d7a/model")
dv = model._model_impl  # Access the DictVectorizer


In [72]:
import mlflow.sklearn
model = mlflow.sklearn.load_model(f"runs:/c94de1a1d7d54fba87e6582023251d7a/model")

In [73]:
print(dir(model))  # See available attributes


['_ElasticNet__metadata_request__fit', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__sklearn_clone__', '__sklearn_tags__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_build_request_for_signature', '_decision_function', '_doc_link_module', '_doc_link_template', '_doc_link_url_param_generator', '_estimator_type', '_get_default_requests', '_get_doc_link', '_get_metadata_request', '_get_param_names', '_get_params_html', '_html_repr', '_parameter_constraints', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_set_intercept', '_validate_params', 'alpha', 'coef_', 'copy_X', 'dual_gap_', 'fit', 'fit_intercept', 'get_metadata_routing', 'get_params

In [62]:
print(type(model))

<class 'mlflow.pyfunc.PyFuncModel'>


In [59]:
print(dv)

In [70]:
import pickle

In [71]:
with open('models/lin_reg2.bin', 'rb') as f_in:
    dv, _ = pickle.load(f_in)
    
X_test = preprocess(df, dv) 

In [74]:
y_pred = model.predict(X_test)

In [75]:
y_pred

array([12.89535573,  9.6594789 , 12.47463313, ..., 14.64493534,
       19.87818932, 21.57954141], shape=(82750,))

In [ ]:
# import mlflow
mlflow.artifacts.download_artifacts(
    run_id="c94de1a1d7d54fba87e6582023251d7a",
    artifact_path="models_pickle/lin_reg2.bin",
    dst_path="./downloaded_models"
)

# from mlflow.tracking import MlflowClient

# client = MlflowClient()
# client.download_artifacts(
#     run_id="your_run_id", 
#     path="models_pickle/lin_reg2.bin",
#     dst_path="./downloaded_models"
# )

'/workspaces/Mlops-Zoomcamp-2025/02-experiment-tracking/downloaded_models/lin_reg2.bin'

In [80]:
with open('./downloaded_models/lin_reg2.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [81]:
X_test = preprocess(df, dv) 

In [82]:
y_pred = model.predict(X_test)

In [83]:
target = "duration"
y_test = df[target].values

In [84]:
%time test_model(run_id="c94de1a1d7d54fba87e6582023251d7a",  X_test=X_test, y_test=y_test)

CPU times: user 58.5 ms, sys: 2.31 ms, total: 60.8 ms
Wall time: 76.5 ms


{'rmse': 8.083311190323899}